# Passivate Slab Surface

Add passivating atoms (e.g., hydrogen) to material surfaces by detecting surface atoms and placing passivants at specific bond lengths.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Material (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click "Run" > "Run All" to run all cells.
1. Wait for the run to complete.
1. Scroll down to view results.

## Notes

1. Surface atoms are detected by analyzing atomic positions along z-axis
1. "Shadowing radius" controls the exclusion of atoms below the surface ones from being passivated
1. Passivants are added at specified bond lengths from surface atoms

## 1. Prepare the Environment
### 1.1. Set up the notebook
Passivation Configuration parameters are described in [Passivation Configuration](https://github.com/Exabyte-io/made/blob/666a941a190a94979b5295f5039f6a4b9158d7dd/src/py/mat3ra/made/tools/build/passivation/configuration.py#L7).

Visualization of the parameters of passivation on the image below:

<img src="https://github.com/Exabyte-io/documentation/raw/88451ce38b0f57f804dcb5010a1a95bb9b9e50bc/images/notebooks/surface_passivation_settings.webp" alt="Passivation parameters" width="600"/>


In [ ]:
# Material selection
MATERIAL_INDEX = 0  # Which material to use from input list

# Passivation parameters
PASSIVANT = "H"  # Chemical symbol of passivating atom
BOND_LENGTH = 1.0  # Distance from surface to passivant, in Angstroms
SURFACE = "top"  # Which surface to passivate: "top", "bottom" or "both"

# Surface detection parameters
SHADOWING_RADIUS = 2.5  # Radius to exclude subsurface atoms, in Angstroms
DEPTH = 5.0  # How deep to look for surface atoms, in Angstroms

BYPASS_SLAB_CREATION = False  # If True, will use input material directly

# Slab parameters for creating a new slab if previous option is set to True
DEFAULT_SLAB_PARAMETERS = {
    "miller_indices": (0, 0, 1),
    "thickness": 3,
    "vacuum": 10.0,
    "USE_ORTHOGONAL_C": True,
    "xy_supercell_matrix": [[3, 0], [0, 3]]
}

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
CELL_REPETITIONS_FOR_VISUALIZATION = [1, 1, 1]  # Structure repeat in view

### 1.2. Install packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install`.

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install(
        "mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("")

### 1.3. Load and preview input material

In [ ]:
from utils.jupyterlite import get_materials
from utils.visualize import visualize_materials

materials = get_materials(globals())

### 1.4. Create a slab if the input material is not a slab

In [ ]:
from mat3ra.made.tools.build.pristine_structures.two_dimensional.slab import SlabConfiguration
from mat3ra.made.tools.helpers import create_slab_if_not
from mat3ra.made.tools.modify import translate_to_z_level

material = materials[MATERIAL_INDEX]
default_slab_config = SlabConfiguration.from_parameters(
    material_or_dict=material,
    miller_indices=DEFAULT_SLAB_PARAMETERS["miller_indices"],
    number_of_layers=DEFAULT_SLAB_PARAMETERS["thickness"],
    vacuum=DEFAULT_SLAB_PARAMETERS["vacuum"],
)

slab = material if BYPASS_SLAB_CREATION else create_slab_if_not(material, default_slab_config)
slab = translate_to_z_level(slab, "center")

if SHOW_INTERMEDIATE_STEPS:
    print("Initial slab structure:")
    visualize_materials(slab, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION)
    visualize_materials(slab, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION, rotation="-90x")


## 2. Create target material
### 2.1. Passivate the slab surface


In [ ]:
from mat3ra.made.tools.helpers import passivate_surface
from mat3ra.made.tools.helpers import SurfaceTypesEnum

passivated_slab = passivate_surface(
    slab,
    passivant=PASSIVANT,
    bond_length=BOND_LENGTH,
    surface=SurfaceTypesEnum[SURFACE.upper()],
    shadowing_radius=SHADOWING_RADIUS,
    depth=DEPTH
)

print("\nPassivation completed:")
print(f"Passivant used: {PASSIVANT}")
print(f"Bond length: {BOND_LENGTH} Å")
print(f"Surfaces passivated: {SURFACE}")
print(f"Number of atoms: {len(passivated_slab.basis.elements.ids)}")

## 3. Visualize the result

In [ ]:
print("Final passivated structure:")
visualize_materials(passivated_slab, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION)
visualize_materials(passivated_slab, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION, rotation="-90x")

### 4. Pass data to the outside runtime


In [ ]:
from utils.jupyterlite import set_materials

set_materials(passivated_slab)